## Ukraine Russia war Twitter entity recognition and sentiment analysis

In [76]:
# import libraries
!pip install spacy
!python -m spacy download en_core_web_md

import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from deep_translator import GoogleTranslator
import re

import os
import csv
import gzip
import pickle

nlp = spacy.load('en_core_web_md')

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 162.5 kB/s eta 0:04:24
     --------------------------------------- 0.0/42.8 MB 326.8 kB/s eta 0:02:11
     --------------------------------------- 0.1/42.8 MB 326.1 kB/s eta 0:02:11
     --------------------------------------- 0.1/42.8 MB 326.1 kB/s eta 0:02:11
     --------------------------------------- 0.1/42.8 MB 326.1 kB/s eta 0:02:11
     --------------------------------------- 0.2/42.8 MB 588.9 kB/s eta 0:01:13
     --------------------------------------- 0.3/42.8 MB 905.4 kB/s eta 0:00:47
     ---------------------------------------- 0.4/42.8 MB 1.0 MB/s eta 0:00:41
     ---------------------------------------- 0.5/42.8 MB 1.2 MB/s eta 0:00:37
     ---------------------------------------- 0.5/42.8 MB 1.

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
# get files
path0='C:/Users/alexd/Documents/ukraine_war_analysis/twitter/20230201_UkraineCombinedTweetsDeduped.csv.gzip'
data0 = pd.read_csv(path0, compression='gzip')

In [47]:
# remove irrelevant columns, show the final data ready for the analysis
data0=pd.DataFrame(data=data0,columns=['username','acctdesc', 'text', 'totaltweets','followers', 'location', 'extractedts'])
data0['text'].dropna()
data0['extractedts'] =  pd.to_datetime(data0['extractedts']).dt.strftime('%d/%m/%Y')
data0.head()

,username,acctdesc,text,totaltweets,followers,location,extractedts
0,1MedyaNews,"News about Kurdistan, greater Middle East. \nM...","📌 ""I consider it absolutely logical that any f...",5311,3192,"Kurdistan, Worldwide",01/02/2023
1,queen_ukraine,NaN,Ukraine War: Inside the battle for Bakhmut\n#c...,14540,46,NaN,01/02/2023
2,isciences,"ISCIENCES, LLC provides expertise on water and...",3-month #water anomalies forecast January 2023...,7972,1301,"Ann Arbor, MI, USA",01/02/2023
3,aaanewsworld,Television Production - News Agency and Video ...,◆F-2戦闘機 F-2A/B \n米国のF-16を、日本の運用の考え方や地理的な特性に合わせ...,341,5,"大阪 Osaka, Tokyo, JAPAN, ASIA",01/02/2023
4,Caribvibetv,#1 TV Show representing the Caribbean Diaspora...,"Friday, Feb 3/23 at 8PM @OakvilleCentre presen...",2716,2553,Canada,01/02/2023


In [ ]:
# translate to english
def rmv_emoji_and_trans_to_en(input):
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    store = []
    for row in input.iterrows():
        row[1][2] = emoji_pattern.sub(r'', row[1][2])
        row[1][2] = GoogleTranslator(source='auto', target='en').translate(row[1][2])
        store.append(row)
    return store

data = rmv_emoji_and_trans_to_en(data0)

In [72]:
# remove stop words
def rem_stopwords(input):
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    store=[]
    for row in input.iterrows():
        doc = nlp(row[1][2])
        for word in doc:
            if word.is_stop==False:
                store.append(word)
        row[1][2] = store
    return store
data_no_stopwords = rem_stopwords(data)

In [ ]:
# sentiment analysis 
# https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis
# https://stackabuse.com/python-for-nlp-introduction-to-the-textblob-library/

# python -m textblob.download_corpora
from textblob import TextBlob
>>> testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
>>> testimonial.sentiment
Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)
>>> testimonial.sentiment.polarity
0.39166666666666666


In [ ]:
# entity recognition


import spacy
  
nlp = spacy.load('en_core_web_md')
  
sentence = "Apple is looking at buying U.K. startup for $1 billion"
  
doc = nlp(sentence)
  
for ent in doc.ents:
    print(ent.text, ent.label_)

final dataset:
- column 1: the tweet
- columns 2: amount of followers
- column 3: total tweets
- column 4: date of the tweet
- column 5: sentiment for the tweet
- column 6 - x: columns each one for an entity type (person, event, etc)

-
- maybe more relevant (location of the tweet?)


In [2]:


file_name = "RussiaUkraineWeiboDataset\scraped_weibo_byUID_comments_filtered.pkl"

with open(file_name, 'rb') as f:
    data = pickle.load(f)